In [3]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math
import geopandas as gpd

In [2]:
df = pd.read_csv("data/us_estdata_full.csv")

In [3]:
migrants = df[df["STAY"] == 0]

In [118]:
flows = migrants.groupby(["ORIGIN", "CHOSEN"])["RT"].count().unstack().fillna(0)

In [259]:
acs_puma = pd.read_csv("data/ACS_2018.csv").set_index("PUMA")
acs_migpuma = pd.read_csv("data/ACS_MIGPUMA_2018.csv").set_index("MIGPUMA")

In [4]:
distances = pd.read_csv("data/puma_distance_matrix_fixed.csv").set_index("Unnamed: 0")
distances.columns = distances.columns.astype(int)

In [5]:
distances

,600105,600102,608502,600108,600107,600101,600110,608504,600109,600106,...,2602200,2600900,4702402,4702300,4702401,4700500,4702501,5310400,5310300,5310100
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
600100,2.811224e+04,2.811224e+04,39057.0,2.811224e+04,2.811224e+04,2.811224e+04,2.811224e+04,32785.0,2.811224e+04,2.811224e+04,...,3659560.0,3658028.0,3704534.0,3707792.0,3697348.0,3709156.0,3675491.0,1510819.0,1366000.0,1521187.0
600700,2.676500e+05,2.541540e+05,308889.0,2.933460e+05,2.845210e+05,2.477670e+05,2.743320e+05,302616.0,2.903600e+05,2.738470e+05,...,3548479.0,3546947.0,3757628.0,3748449.0,3750726.0,3741417.0,3716148.0,1293877.0,1149057.0,1304245.0
601100,3.420630e+05,3.285670e+05,383302.0,3.677590e+05,3.589340e+05,3.221800e+05,3.618530e+05,377029.0,3.647730e+05,3.482590e+05,...,3646878.0,3645346.0,3856026.0,3846847.0,3849125.0,3839816.0,3814546.0,1246358.0,1101538.0,1256725.0
601300,5.418500e+04,4.434600e+04,90212.0,8.001100e+04,6.752000e+04,4.544600e+04,6.876300e+04,83940.0,7.168300e+04,5.517000e+04,...,3635231.0,3633699.0,3743085.0,3746342.0,3735899.0,3747707.0,3714042.0,1486490.0,1341670.0,1496858.0
601500,4.872780e+05,4.737820e+05,528517.0,5.129740e+05,5.041490e+05,4.673950e+05,5.070680e+05,522244.0,5.099870e+05,4.934740e+05,...,3594176.0,3592644.0,3803325.0,3794145.0,3796423.0,3787114.0,3761844.0,1149082.0,1004262.0,1159450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304007,3.996091e+06,4.004766e+06,3970737.0,3.980928e+06,3.987472e+06,4.012807e+06,3.958553e+06,3961958.0,3.991797e+06,3.985390e+06,...,1210028.0,1269788.0,373179.0,429406.0,391176.0,457262.0,418124.0,4143960.0,4251644.0,4527073.0
5151000,4.385759e+06,4.394434e+06,4360405.0,4.370596e+06,4.377141e+06,4.402476e+06,4.348221e+06,4351626.0,4.381466e+06,4.375058e+06,...,917732.0,1041456.0,718843.0,663499.0,695569.0,705881.0,692139.0,4231226.0,4338910.0,4661344.0
5151001,4.543694e+06,4.530198e+06,4542272.0,4.569390e+06,4.560565e+06,4.523811e+06,4.547511e+06,4533493.0,4.566404e+06,4.549890e+06,...,1050887.0,1169711.0,900710.0,845366.0,877436.0,887748.0,874006.0,4350356.0,4458040.0,4780474.0


In [361]:
def loss(x):
    utils = np.zeros((975, 2336))
    for i, migpuma in enumerate(flows.index):
        utils[i, :] = np.log(distances.loc[migpuma, flows.columns]) * x[0] + np.log(acs_puma.loc[flows.columns, "Total Population"].values) * x[1]
    probs = np.exp(utils)
    probs = probs / probs.sum(axis=1)[:, None]
    probs = probs * flows.sum(axis=1).values.reshape(-1, 1)
    return ((flows - probs) ** 2).sum().sum() / (flows.shape[0] * flows.shape[1])

In [324]:
import scipy.optimize

In [355]:
loss([-1, -1])

(975, 2336)


6.075872009134831

In [356]:
scipy.optimize.minimize(loss, x0=[0, 0])

(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)
(975, 2336)


      fun: 2.3257503249501728
 hess_inv: array([[0.34850984, 0.16994061],
       [0.16994061, 2.35960186]])
      jac: array([-5.96046448e-08, -5.96046448e-08])
  message: 'Optimization terminated successfully.'
     nfev: 33
      nit: 7
     njev: 11
   status: 0
  success: True
        x: array([-2.33570142,  0.91851201])

In [362]:
utils = np.zeros((975, 2336))
for i, migpuma in enumerate(flows.T.columns):
    utils[i, :] = np.log(distances.loc[migpuma, flows.T.index]) * -2.33570142 + np.log(acs_puma.loc[flows.T.index, "Total Population"].values) * 0.91851201
probs = np.exp(utils)
probs = probs / probs.sum(axis=1)[:, None]
probs = probs * flows.sum(axis=1).values.reshape(-1, 1)

In [365]:
temp = pd.DataFrame(probs)

In [369]:
((flows.values - temp.values) ** 2).sum().sum() / (975 * 2336)

5297128.94010651

In [3]:
acs_puma_2017 = pd.read_csv('data/ACS_2017_new.csv').set_index("PUMA")
acs_migpuma_2017 = pd.read_csv("data/ACS_MIGPUMA_2017_new.csv").set_index("MIGPUMA")

In [2]:
df_2017 = pd.read_csv('data/us_estdata_2017.csv')

In [4]:
migrants_2017 = df_2017[df_2017["STAY"] == 0]

In [5]:
flows_2017 = migrants_2017.groupby(["ORIGIN", "CHOSEN"])["RT"].count().unstack().fillna(0)
flows_2017

CHOSEN,100100,100200,100301,100302,100400,100500,100600,100700,100800,100900,...,5555102,5555103,5570101,5570201,5570301,5600100,5600200,5600300,5600400,5600500
ORIGIN,,,,,,,,,,,,,,,,,,,,,
100190,123.0,5.0,0.0,4.0,0.0,4.0,6.0,8.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100290,10.0,79.0,62.0,111.0,9.0,100.0,16.0,3.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100400,4.0,6.0,0.0,0.0,77.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100600,4.0,4.0,5.0,1.0,3.0,1.0,67.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100700,5.0,0.0,0.0,0.0,0.0,14.0,3.0,71.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5600100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60.0,3.0,5.0,2.0,3.0
5600200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,55.0,6.0,11.0,0.0
5600300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,5.0,116.0,3.0,5.0


In [14]:
acs_migpuma_2017

,Unnamed: 0,Geo_FIPS,Geo_LOGRECNO,Geo_STATE,Geo_PUMA5,Total Population,Population Density (Per Sq. Mile),Area (Land),Area Total,Area Total Area (Land),...,Civilian Population 18 Years and Over Veteran,Civilian Population 18 Years and Over Veteran 18 to 64 Years,Civilian Population 18 Years and Over Veteran 65 Years and Over,Civilian Population 18 Years and Over Nonveteran,Civilian Population 18 Years and Over Nonveteran 18 to 64 Years,Civilian Population 18 Years and Over Nonveteran 65 Years and Over,Total Population Native Born,Total Population Foreign Born,Total Population Foreign Born Naturalized Citizen,Total Population Foreign Born Not a Citizen
MIGPUMA,,,,,,,,,,,,,,,,,,,,,
100190,0,201500,128,2,1500,302838,64.984558,4660.153273,4773.22600,4660.15300,...,18905,9293,9612,219119,170114,49005,294519.0,8319.0,2862.0,5457.0
100290,1,401303,231,4,1303,550996,285.878530,1927.378036,2043.06821,1927.37761,...,44892,28023,16869,380437,313681,66756,520798.0,30198.0,13427.0,16771.0
100400,2,100400,59,1,400,123526,66.592198,1854.962059,1905.45600,1854.96200,...,7252,3284,3968,88605,70768,17837,116336.0,7190.0,1938.0,5252.0
100600,3,100600,61,1,600,151867,119.576756,1270.037798,1316.25200,1270.03800,...,9397,5205,4192,108234,86000,22234,146475.0,5392.0,1383.0,4009.0
100700,4,100700,62,1,700,106477,78.996099,1347.876688,1386.88900,1347.87700,...,5688,2876,2812,77805,60824,16981,104424.0,2053.0,559.0,1494.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7200900,1000,14401803,153,144,1803,292913,2475.646117,118.317799,180.31621,118.31780,...,7691,3067,4624,227340,174187,53153,279723.0,13190.0,6855.0,6335.0
7201001,1001,7201001,78,72,1001,129604,2211.557194,58.603051,59.06752,58.60305,...,3473,1271,2202,100602,77363,23239,127708.0,1896.0,798.0,1098.0
7201002,1002,7201002,79,72,1002,161248,1140.658831,141.363917,142.02710,141.36390,...,2092,974,1118,123930,98243,25687,0.0,0.0,0.0,0.0


In [13]:
acs_migpuma_2017["Total Population"].values * 0.03

array([ 9085.14, 16529.88,  3705.78, ...,  4837.44,  3654.09,  3390.42])

In [18]:
utils = np.zeros((975, 2336))
for i, migpuma in enumerate(flows_2017.index):
    utils[i, :] = np.log(distances.loc[migpuma, flows_2017.columns]) * -2.33570142 + np.log(acs_puma_2017.loc[flows_2017.columns, "Total Population"].values) * 0.91851201
probs = np.exp(utils)
probs = probs / probs.sum(axis=1)[:, None]
probs = probs * (acs_migpuma_2017.loc[flows_2017.index, "Total Population"].values).reshape(-1, 1) * 0.03

In [21]:
temp = pd.DataFrame(probs)

In [23]:
((probs - flows_2017) ** 2).sum().sum() / (975 * 2336)

6944.198484227278